In [ ]:
# Imports
import os
import pandas as pd
from ipywidgets import *
import numpy as np
import mne
from mne.preprocessing import ICA

# import torch
from mne_icalabel import label_components

import matplotlib.pyplot as plt
from mne.preprocessing import ICA
import scipy
from scipy.interpolate import interp1d

import preprocessing
import utils

# Specify graph rendering method
# %matplotlib widget
plt.switch_backend("TkAgg")

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none
Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


In [2]:
DATASET_PATH = "./dataset"
FILENAME_TEMPLATE = "TMS-EEG-H_02_S1b_{}_{}.vhdr"

# rsEEG_pre_raw = mne.io.read_raw_brainvision(os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("rsEEG", "pre")), preload=True)
spTEP_pre_raw = mne.io.read_raw_brainvision(
    os.path.join(DATASET_PATH, FILENAME_TEMPLATE.format("spTEP", "pre")), preload=True
)
sampling_rate = spTEP_pre_raw.info["sfreq"]
events, event_dict = mne.events_from_annotations(spTEP_pre_raw)
tms_indices = [event[0] for event in events if event[2] == 1]

Extracting parameters from ./dataset\TMS-EEG-H_02_S1b_spTEP_pre.vhdr...
Setting channel info structure...
Reading 0 ... 2696199  =      0.000 ...   539.240 secs...
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']


In [3]:
# Plotting utilities
def plot_single_response(eeg_data, channel="Pz", tmin=-0.005, tmax=0.01):
    events, event_dict = mne.events_from_annotations(eeg_data)
    event_id = event_dict["Stimulus/S  1"]
    epochs = mne.Epochs(
        eeg_data,
        events,
        event_id=event_id,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        preload=True,
        picks=channel,
    )

    epochs.plot(picks=channel, n_epochs=1, show=True, scalings={"eeg": 50e-4})


def plot_average_epoch(epochs, start=-0.05, end=0.25):
    data = epochs.get_data()
    mean_responses = np.mean(data, axis=0)
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    for i, mean_response in enumerate(mean_responses):
        selected_data = mean_response[selected_indices]
        selected_time_points = time_points[selected_indices]
        plt.plot(selected_time_points, selected_data, label=f"Channel {i+1}")
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.show()


def plot_response(eeg):
    utils.plot_average_response(eeg, tmin=-0.05, tmax=0.25)  # Check full response
    utils.plot_single_response(
        eeg, channel="Pz", tmin=-0.05, tmax=0.05
    )  # Check for TMS pulse

# Cleaning

The paper of Bertazzoli et al. (2021) compares 4 pipelines: ARTIST, TMSEEG, TESA and SOUND-SSP-SIR, all of which work decently well in varying degrees. There are common steps, but TESA will be the one that will be most closely followed. The current steps are as follows:

1. Remove EOG
2. Remove TMS pulse
3. Downsample
4. **ICA - 1**
5. Bandpass - Notch filters
6. **ICA - 2**
7. Rereference

Currently, there is no demeaning or bad channel rejection present as in TESA. Demeaning is done before the TMS-pulse interpolation, and baseline correction should be done as last step after rereferencing.


In [4]:
plot_response(spTEP_pre_raw)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 1251 original time points ...


1 bad epochs dropped
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 501 original time points ...
0 bad epochs dropped
Using matplotlib as 2D backend.


![](img/raw-average.png)
![](img/single-pulse-raw.JPG)

The figure above already shows 2 major artifacts that cleaning will have the biggest impact on: TMS-pulse interpolation and demeaning/baseline correction. The major TMS-pulse artifact falls closely within the range of 2ms before until 5ms after the pulse.


In [4]:
spTEP_copy = spTEP_pre_raw.copy()

## EOG removal


In [5]:
def remove_EOG(eeg_data):
    eeg_data.drop_channels(["HEOG", "VEOG"])

In [6]:
remove_EOG(spTEP_copy)

## TMS pulse removal


In [7]:
def calculate_range_indices(tms_index, start, end, sampling_rate):
    """
    start and end are positive in seconds
    sampling rate in Hz
    """
    samples_before = int(start * sampling_rate)
    samples_after = int(end * sampling_rate)

    start_index = max(0, tms_index - samples_before)
    end_index = tms_index + samples_after

    return start_index, end_index

In [8]:
def interpolate_TMS_pulse(eeg_data_raw, tms_indices, start, end, sampling_rate):
    eeg_data = eeg_data_raw.get_data()
    num_electrodes = eeg_data.shape[0]
    for tms_index in tms_indices:
        start_index, end_index = calculate_range_indices(
            tms_index, start, end, sampling_rate
        )
        for i in range(num_electrodes):
            x = [start_index - 2, start_index - 1, end_index + 1, end_index + 2]
            y = [
                eeg_data[i, start_index - 2],
                eeg_data[i, start_index - 1],
                eeg_data[i, end_index + 1],
                eeg_data[i, end_index + 2],
            ]
            x_new = np.arange(start_index, end_index + 1)

            interp_func = interp1d(x, y, kind="cubic")
            eeg_data[i, start_index : end_index + 1] = interp_func(x_new)

    eeg_data_raw._data = eeg_data

In [9]:
interpolate_TMS_pulse(
    spTEP_copy, tms_indices, 0.002, 0.005, sampling_rate
)  # 2ms before, 5ms after

In [10]:
plot_response(spTEP_copy)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found


No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 1501 original time points ...
1 bad epochs dropped
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 501 original time points ...
0 bad epochs dropped
Using matplotlib as 2D backend.


![](img/interpolated-average.png)
![](img/single-pulse-interpolated.JPG)


This succesfully removed the TMS pulse artifact by using cubic interpolation based on the 2 values before and 2 values after the range that is to be interpolated.


## Downsampling

The original data was captured with a sampling frequency of 5000 Hz. 1000 Hz is chosen as the frequency to be downsampled to, as this means that, following Nyquists theorem, the highest frequency that will be accurately recorded is 500 Hz, which should be more than enough for further analysis, as the gamma band is often referred to as 30-100 Hz.


In [11]:
def downsample(eeg_data, sample_rate=1000):
    eeg_data.resample(sample_rate, npad="auto", n_jobs=-1)

In [12]:
downsample(spTEP_copy)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:   16.6s finished


In [13]:
plot_response(spTEP_copy)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 301 original time points ...
1 bad epochs dropped
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 101 original time points ...
0 bad epochs dropped


![](img/single-downsampled.JPG)


## Epoching


In [14]:
def epoching(eeg_data):
    events, event_dict = mne.events_from_annotations(eeg_data)
    event_id = event_dict["Stimulus/S  1"]
    epochs = mne.Epochs(
        eeg_data,
        events,
        event_id=event_id,
        tmin=-1,
        tmax=1,
        baseline=None,
        preload=True,
    )
    return epochs

In [15]:
epochs = epoching(spTEP_copy)

Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 2001 original time points ...
0 bad epochs dropped


In [16]:
plot_average_epoch(epochs)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch.png)


## Demeaning/detrending

Demeaning is achieved by subtracting each value from each electrode with the average value of the corresponding electrode, essentially bringing the means from all electrodes to 0.

> TODO: check if other way of demeaning on complete electrode is possible to move value near 0 or better yet on 0


In [17]:
def demean(eeg_data):
    eeg_data.apply_function(lambda x: x - x.mean())

In [18]:
demean(spTEP_copy)

In [19]:
data = spTEP_copy.get_data()
mean_values = np.mean(data, axis=1)
total_mean = np.mean(mean_values)
print(f"Total mean: {total_mean}")

plot_response(spTEP_copy)

Total mean: 9.97476064716104e-20
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
151 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 151 events and 301 original time points ...
1 bad epochs dropped
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1']
Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 150 events and 101 original time points ...
0 bad epochs dropped


In [20]:
def demean_epochs(epochs):
    data = epochs.get_data()
    demeaned_data = data - np.mean(data, axis=2, keepdims=True)
    demeaned_epochs = mne.EpochsArray(
        demeaned_data, epochs.info, events=epochs.events, event_id=epochs.event_id
    )
    return demeaned_epochs

In [21]:
epochs = demean_epochs(epochs)

Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated


C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\3590353875.py:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


In [22]:
plot_average_epoch(epochs)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


Below are two graphs with demeaning applied to both the full electrode and to individual epochs. Clearly, there has to be some error in demeaning the full electrode, as those averages aren't close enough to 0. However, when printing the mean values of the electrodes they are indeed close to 0 (e.g. 1\*e-20). For now, as the demeaned epochs are the desired result, the epochs will be used for further cleaning.

![](img/average-demean.png)
![](img/average_epoch_demeaned.png)


## ICA - 1

The first ICA filter is mainly to remove the primary large artifacts such as muscle and electrical charge. If demeaning were applied now, a graph as below is the result.

This is implemented by first fitting ICA to the signal, and then applying the threshold formula used by the TESA software to each component to either keep or remove each ICA component.


In [23]:
def ICA_1(epoch_data, T=3.5, b1=0.011, b2=0.030, n_components=20):
    ica = ICA(n_components=n_components, random_state=97)
    ica.fit(epoch_data)

    # Credits to Arne Callaert for the following code
    sources = ica.get_sources(epoch_data)
    averaged_sources = sources.get_data().mean(axis=0)
    times = sources.times
    sfreq = sources.info["sfreq"]
    indices = np.where((times >= (b1 / 1000)) & (times <= (b2 / 1000)))
    print("indices:", indices)
    components_to_remove = []

    for i, component in enumerate(averaged_sources):
        base = len(times) / 2
        b1_index = int(base + (b1 * sfreq))
        b2_index = int(base + (b2 * sfreq))
        x = np.mean(np.abs(component[b1_index:b2_index]))
        y = np.mean(np.abs(component))
        if x / y > T:
            print("FOUND:", x / y)
            components_to_remove.append(i)

    ica.exclude = components_to_remove

    epoch_data = ica.apply(epoch_data)

In [24]:
ICA_1(epochs)

Fitting ICA to data using 62 channels (please be patient, this may take a while)


C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\1222509230.py:3: RuntimeWarning: The data has not been high-pass filtered. For good ICA performance, it should be high-pass filtered (e.g., with a 1.0 Hz lower bound) before fitting ICA.
  ica.fit(epoch_data)


Selecting by number: 20 components
Fitting ICA took 11.8s.
indices: (array([], dtype=int64),)
FOUND: 4.284637249273379
FOUND: 4.326392494594478
Applying ICA to Epochs instance
    Transforming to ICA space (20 components)
    Zeroing out 2 ICA components
    Projecting back using 62 PCA components


C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\1222509230.py:7: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  averaged_sources = sources.get_data().mean(axis=0)


In [25]:
plot_average_epoch(epochs)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch_ica_1.png)
The graph below shows the result after applying the ICA filter. However, this looks like it results in muddier values than before. The goal was to filter out the initial peak values, residue from the TMS pulse, but these are still present.


## Bandpass - Notch


In [26]:
def bandpass_notch(epoch_data, low_freq=0.5, high_freq=100, notch_freqs=[50]):
    # Bandpass
    epoch_data.filter(low_freq, high_freq)

    # Notch (only directly available on raw object, not on epochs)
    data = epoch_data.get_data()
    notch_filtered = mne.filter.notch_filter(data, epochs.info["sfreq"], notch_freqs)
    filtered_epochs = mne.epochs.EpochsArray(
        notch_filtered, epochs.info, events=epochs.events, tmin=epochs.tmin
    )

    return filtered_epochs

In [27]:
epochs = bandpass_notch(epochs)

Setting up band-pass filter from 0.5 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 6601 samples (6.601 s)



C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\1507634377.py:3: RuntimeWarning: filter_length (6601) is longer than the signal (2001), distortion is likely. Reduce filter length or filter a longer signal.
  epoch_data.filter(low_freq, high_freq)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.6s
[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.7s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.9s
[Parallel(n_jobs=1)]: Done 2177 tasks      | elapsed:    1.1s
[Parallel(n_jobs=1)]: Done 2591 tasks      | elapsed:    1.3s
[Parallel(n_jo

Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 6601 samples (6.601 s)



C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\1507634377.py:6: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epoch_data.get_data()
C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\1507634377.py:7: RuntimeWarning: filter_length (6601) is longer than the signal (2001), distortion is likely. Reduce filter length or filter a longer signal.
  notch_filtered = mne.filter.notch_filter(data, epochs.info["sfreq"], notch_freqs)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done 1151 tasks      | e

Not setting metadata
150 matching events found
No baseline correction applied
0 projection items activated


In [28]:
plot_average_epoch(epochs)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch_filter.png)


## ICA - 2


In [29]:
def ICA_2(epoch_data):
    ica = mne.preprocessing.ICA(n_components=20, random_state=42)
    ica.fit(epoch_data)
    muscle_idx_auto, scores = ica.find_bads_muscle(epoch_data)
    print(muscle_idx_auto, scores)
    ica.exclude = muscle_idx_auto
    epoch_data = ica.apply(epoch_data)

In [30]:
epochs_copy = epochs.copy()

In [ ]:
ICA_2(epochs_copy)
plot_average_epoch(epochs_copy)

In [36]:
ICA_2(epochs_copy)
plot_average_epoch(epochs_copy)

Fitting ICA to data using 62 channels (please be patient, this may take a while)


Selecting by number: 20 components
Fitting ICA took 11.8s.
    Using multitaper spectrum estimation with 7 DPSS windows
[4, 7, 11, 12, 14, 16, 17, 18, 19] [1.28538003e-02 6.86887319e-03 8.90673843e-06 2.04677528e-05
 2.83462359e-01 3.38681922e-03 6.30374050e-02 1.37735817e-01
 3.28773993e-03 6.85376573e-02 8.72962794e-02 2.71849285e-01
 6.17611817e-01 1.39803389e-03 5.96980972e-01 5.07959846e-02
 3.69729254e-01 7.10909001e-01 6.27275958e-01 5.26146826e-01]
Applying ICA to Epochs instance
    Transforming to ICA space (20 components)
    Zeroing out 9 ICA components
    Projecting back using 62 PCA components


By accident, the second round of ICA was applied twice. However, this resulted in the two graphs shown below. Clearly, the second one practically removed the artifact that round one and two were supposed to remove. However, it isn't clear yet why it requires the second round to be applied twice.


![](img/average_epoch_ica_2.png)
![](img/average_epoch_ica_2_2.png)


## Rereference


In [38]:
def rereference(epochs):
    mne.set_eeg_reference(epochs, ref_channels="average")

In [39]:
rereference(epochs_copy)

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


In [40]:
plot_average_epoch(epochs_copy)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch_rereference.png)


## Baseline correction


In [41]:
def baseline(epoch_data):
    epoch_data.apply_baseline((None, None))

In [42]:
baseline(epochs_copy)

Applying baseline correction (mode: mean)


In [43]:
plot_average_epoch(epochs_copy)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\696622896.py:20: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch_baseline.png)


In [44]:
def plot_full_average_epoch(epochs, start=-0.05, end=0.2):
    data = epochs.get_data()
    mean_responses = np.mean(data, axis=(0, 1))
    sem_responses = np.std(data, axis=(0, 1)) / np.sqrt(data.shape[0])
    time_points = np.linspace(-1, 1, data.shape[2])
    selected_indices = np.where((time_points >= start) & (time_points <= end))
    selected_data = mean_responses[selected_indices]
    selected_sem = sem_responses[selected_indices]
    selected_time_points = time_points[selected_indices]
    plt.plot(selected_time_points, selected_data, label="Average of all electrodes")
    plt.fill_between(
        selected_time_points,
        selected_data - selected_sem,
        selected_data + selected_sem,
        color="b",
        alpha=0.2,
    )
    plt.xlabel("Time points")
    plt.ylabel("Mean response")
    plt.legend()
    plt.show()

## Final result

In these final plots, the total average is plotted with the error range.


In [45]:
plot_full_average_epoch(epochs_copy)

C:\Users\Tomas\AppData\Local\Temp\ipykernel_32460\352732209.py:2: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  data = epochs.get_data()


![](img/average_epoch_all_electrodes.png)


In [ ]:
# Save the epochs to a file
epochs_copy.save("processed_epochs-epo.fif")

## TODO

Current biggest things to find out:

- is there a way to further improve the filtering that ICA 1 is supposed to achieve? (Filtering out the residue of the TMS pulse)
- how can time ranges be plot on the scalp topography? like in the comparative paper
